<a href="https://colab.research.google.com/github/abar-1/SDR-ML-Project/blob/editNN/QAMReceiverV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Function to convert binary data to QAM16 constellations

In [7]:
pip install tensorflow

16QAM Constellation Modulator

In [14]:
import numpy as np
"""
Used https://dsplog.com/2008/06/01/binary-to-gray-code-for-16qam/ for mappings. When tested, the function works and the binary is correctly mapped to its corresponding
complex number based on the constellation. To check, uncomment the last line in the cell which prints a test run of the function.
"""


def modulator(binary_data, M):
    k = int(np.log2(M))

    # defining the real and imaginary PAM constellation for 16-QAM
    alphaRe = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)
    alphaIm = np.arange(-(2*np.sqrt(M)/2-1), (2*np.sqrt(M)/2), 2)

    # taking b0b1 for real
    ipDecRe = np.array([int(''.join(map(str, b[:k//2])), 2) for b in binary_data])
    ipGrayDecRe = ipDecRe ^ (ipDecRe >> 1)

    # taking b2b3 for imaginary
    ipDecIm = np.array([int(''.join(map(str, b[k//2:])), 2) for b in binary_data])
    ipGrayDecIm = ipDecIm ^ (ipDecIm >> 1)

    # mapping the Gray coded symbols into constellation
    modRe = alphaRe[ipGrayDecRe]
    modIm = alphaIm[ipGrayDecIm]

    # complex constellation
    mod = modRe + 1j * modIm

    return mod

def generate_qam_symbols(M=16, num_symbols=1000):
    # 4 bits per symbol
    k = int(np.log2(M))

    # Generate random binary data
    random_bits = np.random.randint(0, 2, num_symbols * k)

    # Reshape to match modulator input
    binary_data = random_bits.reshape((-1, k))

    # Modulate Binary Data
    modulated_symbols = modulator(binary_data, M)

    return modulated_symbols, binary_data
#print(generate_qam_symbols(16,5))

(array([-1.+3.j,  1.-1.j,  1.+3.j,  3.+3.j, -3.+3.j]), array([[0, 1, 1, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 0, 1, 0],
       [0, 0, 1, 0]]))


Adding noise to 16QAM signals

In [90]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def add_awgn(qam_symbols, snr_db):
    """
    Adds Additive White Gaussian Noise (AWGN) to QAM symbols.

    Parameters:
        qam_symbols (numpy array): The transmitted QAM symbols (complex numbers).
        snr_db (float): Signal-to-noise ratio in dB.

    Returns:
        numpy array: Noisy QAM symbols.
    """
    # Calculate signal power
    signal_power = np.mean(np.abs(qam_symbols) ** 2)

    # Compute noise power based on SNR (convert dB to linear scale)
    noise_power = signal_power / (10 ** (snr_db / 10))

    # Generate AWGN noise
    noise = np.sqrt(noise_power / 2) * (np.random.randn(*qam_symbols.shape) + 1j * np.random.randn(*qam_symbols.shape))

    # Add noise to the symbols
    noisy_qam_symbols = qam_symbols + noise
    return noisy_qam_symbols


# Generate 16QAM symbols
M = 16
num_symbols = 1000
qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)

#Signal to noise ratio in dB
#High signal to noise ratio is good, low is bad
snr_db = 15


# Add noise to QAM symbols
noisy_qam = add_awgn(qam_symbols, snr_db)

df = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
df.reset_index(inplace=True)

df.drop(['index'],axis=1,inplace=True)

#Getting data of different noise levels (data augmentation to prevent overfitting)
for i in range(15, 55, 5):
  M = 16
  num_symbols = 2000
  qam_symbols, original_binary = generate_qam_symbols(M, num_symbols)
  snr_db = i
  noisy_qam = add_awgn(qam_symbols, snr_db)
  df1 = pd.DataFrame({'features':original_binary.tolist(), 'target':noisy_qam.tolist()})
  df1.reset_index(inplace=True)
  df1.drop(['index'],axis=1,inplace=True)
  df = pd.concat([df, df1])

  #Plot Constellations (Before and After Noise)
  # plt.figure(figsize=(10,5))
  # plt.subplot(1,2,2)
  # plt.scatter(noisy_qam.real, noisy_qam.imag, alpha=0.5, label="Noisy")
  # plt.title(f"Signal to Noise Ratio = {snr_db} dB)")
  # plt.xlabel("In-phase (I)")
  # plt.ylabel("Quadrature (Q)")
  # plt.grid()

  #plt.show()
df = df.rename(columns={'features': 'binary','target':'complex'})
#To save as CSV
df.to_csv('dataWithNoise.csv', index = False)




Build Neural Network

X = Complex

y = Binary converted to Decimal

In [89]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

#Target variable (binary data)
y = df['binary'].values
print(y[:5])
#Convert y from binary to decimal values
decimal_array = []
for binary_number in y:
  decimal_value = 0
  for i, bit in enumerate(reversed(binary_number)):
    decimal_value += bit * (2**i)
  decimal_array.append(decimal_value)
y = np.array(decimal_array)
#Testing to see if conversion is correct, it is
print(y[:5])


# Features (complex data)
X = df['complex'].values
X = np.array([np.array(xi) for xi in X])

# Reshape to (n_samples, 2) for real and imaginary parts
X = [[x.real, x.imag] for x in X]
X = np.array(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(X_train)
print(y_train)
# Build the model
model = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    BatchNormalization(),
    Dropout(0.3),

    Dense(32, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(16, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),

    Dense(1, activation='linear')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")



[list([0, 1, 0, 1]) list([1, 1, 1, 0]) list([1, 1, 1, 0])
 list([0, 1, 1, 1]) list([0, 0, 0, 0])]
[ 5 14 14  7  0]
[[-3.22934144  2.99795137]
 [-0.99280508 -1.00464553]
 [ 1.06486763  3.0417588 ]
 ...
 [-3.00713923  3.00824455]
 [-2.70176009  0.4389824 ]
 [-0.91505457 -2.96105393]]
[ 2  5 14 ...  2  3  4]
Epoch 1/10


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(32, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


820/820 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.0639 - loss: 8.9387e-07 - val_accuracy: 0.0747 - val_loss: 8.9572e-07
Epoch 2/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0654 - loss: 8.8703e-07 - val_accuracy: 0.0845 - val_loss: 8.9572e-07
Epoch 3/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0655 - loss: 8.8796e-07 - val_accuracy: 0.0913 - val_loss: 8.9572e-07
Epoch 4/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.0646 - loss: 8.9608e-07 - val_accuracy: 0.0796 - val_loss: 8.9572e-07
Epoch 5/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0697 - loss: 8.8573e-07 - val_accuracy: 0.1063 - val_loss: 8.9572e-07
Epoch 6/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.0654 - loss: 8.9162e-07 - val_accuracy: 0.1037 - val_loss: 8.9572e-07
Epoch 7/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.0641 - loss: 8.9127e-07 - val_accuracy: 0.0910 - val_loss: 8.9572e-07
Epoch 8/10
820/820 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accur

/usr/local/lib/python3.11/dist-packages/keras/src/losses/losses.py:33: SyntaxWarning: In loss categorical_crossentropy, expected y_pred.shape to be (batch_size, num_classes) with num_classes > 1. Received: y_pred.shape=(None, 1). Consider using 'binary_crossentropy' if you only have 2 classes.
  return self.fn(y_true, y_pred, **self._fn_kwargs)


257/257 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.1128 - loss: 8.8562e-07
Test Accuracy: 11.05%


Make NN with last layer w/ 1 neuron

Use same training data

Save Train and Test as CSV



Stage 1: Put the last layer as linear activation, map to 0-15

Stage 2: See if we can use decision trees (what accuracy? how does it compare?)

Stage 3: Increase size of dataset to 20k

In [86]:
X_pred = np.array([[3, -3],[1, -3],[3, -1],[-1, 3]])  # Convert X_pred to a NumPy array

predictions = model.predict(X_pred)
print(f"Predictions: {predictions}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
[[1.]
 [1.]
 [1.]
 [1.]]
